In [2]:
# import the necessary packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import cv2
import os


## Modules to check the performance of the code
from time import process_time 
# !pip install memory_profiler ## When running from Google Colab
# import memory_profiler as mem_profile
# print('Memory (Before): {}Mb'.format(mem_profile.memory_usage()))


## Importing the necessary TesnorFLow modules modules
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# from tensorflow.python.compiler.mlcompute import mlcompute
# mlcompute.set_mlc_device(device_name='gpu')

from tensorflow import keras
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from sklearn.metrics import r2_score ## form calcualting the r2 score
from sklearn.utils import shuffle
from tensorflow import keras as k
from keras_preprocessing.image import ImageDataGenerator

## Trasfer Learning Models
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1 
from tensorflow.keras.applications import EfficientNetB2, EfficientNetB3
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB5 
from tensorflow.keras.applications import EfficientNetB6, EfficientNetB7
from tensorflow.keras.applications import ResNet50,ResNet50V2

In [ ]:
############ Please provide the same path to the code directory if using Colab################

# Path_gdrive= '/content/drive/MyDrive/DPNNet-RT/' ## Comment out this line if using local computer

## Importing the Modules from Modules_DPNNet
import sys
try: ## tries to find the modules in the local directory first
  current_directory = os.getcwd()
  path = current_directory + '/' # For local computer 
#   path = '' # For local computer  
  sys.path.append(path+'MODULES_DPNNeT')
  import data_processing_RT as dp
  import deep_models as dm
  import other_cnns as ocn

########### Folders to save the processed data, files and figures when using Local computer ##############
  output_folder_list = ['data_folder','figures','saved_model']
  for file in output_folder_list:
    try:
        os.makedirs(file)
    except OSError:
        print ("Creation of the directory %s failed/ not needed as it already exit" % file)
    else:
        print ("Successfully created the directory %s" % file)
  
except ModuleNotFoundError:
  
  # #For Colab use:
  # #Point to the path containing the modules in the above section
  #(data folder are a directory above the directory containing the notebook)
  try:
    path = Path_gdrive
    print(path)
    sys.path.append(path+'MODULES_DPNNeT')
    import data_processing_RT as dp
    import deep_models as dm
    import other_cnns as ocn

    ########### Folders to save the processed data, files and figures when using GDRIVE ##############
    import os
    os.chdir(path)
    print("Creating the folders")
    !mkdir -p data_folder
    !mkdir -p figures ## to save the figures
    !mkdir -p figures_paper
    !mkdir -p saved_model
  except ModuleNotFoundError:
    print("The path to the modules is incorrect-- Provide current path")

print("[INFO] Modules imported")



# Model Selections and Hyperparameters

In [ ]:

#### Desired Image resoltuion  and Batch Size #####
X_res = Y_res = 128


## Select the kind of Traning ## Both can be selected as well
REG = True #True  ## When choosing regression
CLA = False #False #False #True  ## When Choosing Clasiffication


## Select the Network type

# NETWORK = "Vanilla" ## Cannot be uses at the moment
# NETWORK = "ALEXNET"
# NETWORK = "VGG"
# NETWORK = "RESNET50"

                                ########## When using Trasnfer Learning ######################
NETWORK = "TL"
transfer_model=ResNet50
# transfer_model=EfficientNetB3

## Hyper-Parameter to define
batch_size = 50 ## 20 was for regression ## the best was for 200 last run

# early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.2, patience=20, verbose=1, mode='min',restore_best_weights=True)


In [ ]:
test = pd.read_csv("data_folder/test_dataset.csv")
train = pd.read_csv("data_folder/train_dataset.csv")
## Generate the Normalized data
normed_train_data, normed_test_data, train_labels, test_labels = dp.process_the_disk_attributes(train, test, path,multi_label=True)

if CLA == True:
        ### Preparing the dataset for the classification problem--- 
        ## For classification we included binary 1 or 0 depending on if the planet is present or not respectively
        train['Planet_Mass1'] = np.where(train['Planet_Mass1']!= 0, 1,0)
        train['Planet_Mass2'] = np.where(train['Planet_Mass2']!= 0, 1,0)
        train['Planet_Mass3'] = np.where(train['Planet_Mass3']!= 0, 1,0)
        # train = train.select_dtypes("string")
        test['Planet_Mass1'] = np.where(test['Planet_Mass1']!= 0, 1, 0)
        test['Planet_Mass2'] = np.where(test['Planet_Mass2']!= 0, 1, 0)
        test['Planet_Mass3'] = np.where(test['Planet_Mass3']!= 0, 1, 0)
        
        Y_train=train[['Planet_Mass1','Planet_Mass2','Planet_Mass3']] ## These are used as label for the non-generator training
        Y_test=test[['Planet_Mass1','Planet_Mass2','Planet_Mass3']]
        

def custom_augmentation(np_tensor):

    '''
    This function is used to crop the images when those are loaded using the 
    ImageDataGenerator Keras function. This custom augmentation function only
    works for three different res as given below. For other resolutions the 
    image needs to the cropped appropiately.

    '''

    # # # dimensions for cropping the image
    if X_res == 128:
      top,left,bottom,right = 20,25,110,90 
    if X_res == 256:          
      top,left,bottom,right = 40,50,220,180          
    if X_res == 512:
      top,left,bottom,right = 60,90,450,380          

    image = np.squeeze(np_tensor) 
    crop_image = image[top:bottom, left:right]
    crop_image = cv2.resize(crop_image, (X_res, Y_res)) 
    crop_image = k.preprocessing.image.img_to_array(crop_image)
    return crop_image


datagen= ImageDataGenerator(preprocessing_function=custom_augmentation,rescale=1./255.,validation_split=0.15)

#### TESTING GENERATOR #####
test_datagen= k.preprocessing.image.ImageDataGenerator(preprocessing_function=custom_augmentation,rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory=None,
x_col="image_path",
y_col=["Planet_Mass1",'Planet_Mass2','Planet_Mass3'],
batch_size=batch_size,
seed=42,
shuffle=False,
class_mode=None,
target_size=(X_res,Y_res))

if X_res < 128:
    testImagesX = dp.load_disk_images(test, X_res, Y_res, Type = "Test")

## STEP SIZES later used for training using generators

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size



## Loading the Model

In [ ]:
# Loading the model

if NETWORK == "TL":
    if REG == True and CLA == True:
        CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str( transfer_model.__name__)+'_'+str(X_res)+'_modelRC')
    elif REG == True:
        CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str( transfer_model.__name__)+'_'+str(X_res)+'_modelR')
    elif CLA == True:
        CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str( transfer_model.__name__)+'_'+str(X_res)+'_modelC')
        
    print("INFO:The Trained model {} at res {} is loaded ".format(NETWORK+'_'+str( transfer_model.__name__),str(X_res)))
else:
    if REG == True and CLA == True:
        CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str(X_res)+'_modelRC')
    elif REG == True:
        CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str(X_res)+'_modelR')
    elif CLA == True:
        CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str(X_res)+'_modelC')



# CNN = tf.keras.models.load_model(path+'saved_model/'+NETWORK+'_'+str(X_res)+'_modelRC')
#Check its architecture
# CNN.summary()

In [ ]:

if X_res >= 128:

    test_generator.reset()
    pred_CNN=CNN.predict_generator(test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
else:
    
    pred_CNN = CNN.predict(testImagesX)  
# pred_CNN[test_index]


In [ ]:
test_index = 41 # 550  #550 ##210
# plt.imshow(testImagesX[test_iclass_mode)
if REG == True:
    print("The predicted Values are {} and \nThe True values are \n{} ".format(pred_CNN[test_index],test_labels.iloc[test_index]))
elif CLA == True:
    print("The predicted prbability of the presence of planets are {} and \nThe True values are \n{} ".format(pred_CNN[test_index],Y_test.iloc[test_index]))
    print("The predicted Values are {} and \nThe True values are \n{} ".format(pred_CNN[test_index],test_labels.iloc[test_index]))

# plt.imshow(testImagesX[test_index])
# test_labels.iloc[test_index]

In [ ]:
plt.style.use('classic')
plt.figure(figsize = (5,5))
# test_predictions = model.predict(normed_test_data).flatten()
plt.scatter(test_labels.to_numpy().flatten(),pred_CNN.flatten(),s=15,marker='d',color='r')
score_CNN = r2_score(test_labels.to_numpy().flatten(),pred_CNN.flatten())
plt.text(20,880,r" r2 = {:.3f}".format(score_CNN), fontsize =14)
plt.xlabel(r'True values of planet mass($M_\oplus$)', fontsize=15)
plt.ylabel(r'Predicted planet mass($M_\oplus$)',fontsize=15)
plt.title("{} Prediction".format(NETWORK))
plt.axis('equal')
plt.axis('square')
plt.xlim(10,1000)
plt.ylim(10,1000)
# plt.yscale("log")
# plt.xscale('log')
# plt.xlim([0.6,plt.xlim()[1]])
# plt.ylim([0.6,plt.xlim()[1]])
_ = plt.plot([0, 1200], [0, 1200],linewidth=2)
plt.minorticks_on() 
plt.tight_layout()
plt.show()
# plt.savefig(path+'figures_paper/predicted_correlation_{}.pdf'.format(NETWORK),format='pdf',dpi=300)

# np.shape(pred_CNN.flatten())